# Linear Solution

In [ ]:
class Linear(Node):
    def __init__(self, x, w, b):
        Node.__init__(self, [x, w, b])

    def forward(self):
        self.cache[0] = np.copy(self.input_nodes[0].value)
        self.cache[1] = np.copy(self.input_nodes[1].value)
        self.cache[2] = np.copy(self.input_nodes[2].value)
        self.value = np.dot(self.cache[0], self.cache[1]) + self.cache[2]

    def backward(self):
        self.dvalues = {n: np.zeros_like(n.value) for n in self.input_nodes}
        if len(self.output_nodes) == 0:
            self.dvalues[self.input_nodes[0]] += np.dot(np.ones_like(self.value), self.cache[1].T)
            self.dvalues[self.input_nodes[1]] += np.dot(self.cache[0].T, np.ones_like(self.value))
            self.dvalues[self.input_nodes[2]] += self.value.shape[0] # equivalent to summing this amount of 1s
            return
        for n in self.output_nodes:
            dval = n.dvalues[self]
            self.dvalues[self.input_nodes[0]] += np.dot(dval, self.cache[1].T)
            self.dvalues[self.input_nodes[1]] += np.dot(self.cache[0].T, dval)
            self.dvalues[self.input_nodes[2]] += np.sum(dval, axis=0, keepdims=True)

## Forward Pass

For the forward pass we compute

$$
f(X, W, b) = XW + b
$$

and store the values in the cache.

## Backward Pass

Note the different initialization here, `np.zeros_like` creates an array the same shape as the input filled with 0s.

```
self.dvalues = {n: np.zeros_like(n.value) for n in self.input_nodes}
```

For the backward pass it's very helpful to write out $f$ explicitly with the elements of the matrices and vector. Let's assume $X$ is a $m$ by $n$ matrix, $W$ is $n$ by $p$, $b$ is $1$ by $p$, and $Z$ is $m$ by $p$.

$$
Z_{ij} = \sum_{k=1}^n X_{ik}W_{kj} + b_j
$$

This is a dot product between the ith row of $X$ and the jth column of $W$ followed by an additon of the jth element of $b$.

Let's find the partial for $X_{ij}$.

$$
\frac {\partial Z_{ij}} {\partial X_{ik}} = W_{kj}
$$

If we take a closer look we'll see that $X_{ik}$ is used to compute more than $Z_{ij}$. Notice the $i$ and $j$ indices remain static. That is, any $X_{ik}$ will be multiplied by an element used in every column $j$ of $W$.

$$
\frac {\partial Z_i} {\partial X_{ik}} = \sum_{j=1}^p W_{kj}
$$

All that's left is to incorporate the values propagated backwards through the chain rule.

$$
\frac {\partial L} {\partial X_{ik}} = \sum_{j=1}^p W_{kj} \frac {\partial L} {\partial Z_{ij}}
$$

In order to express this in code we need to transpose $W$ so it's $p$ by $n$.

$$
\frac {\partial L} {\partial X} = ZW^T
$$

Similar reasoning leads to 

$$
\frac {\partial L} {\partial W} = X^TZ
$$

Ok, now let's find the partial for $b_j$

$$
\frac {\partial Z_{ij}} {\partial b_j} = 1
$$

Once again note we use the same $b_j$ to compute each row $i$ of $Z$.

$$
\frac {\partial Z_{j}} {\partial b_j} = \sum_{i=1}^m 1 = m
$$

Incorporating the chain rule

$$
\frac {\partial L} {\partial b_{j}} = \sum_{i=1}^m (1 * \frac {\partial L} {\partial Z_{ij}})
$$